In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../..")

import os

os.chdir("../..")

print(os.getcwd())

import logging

from datetime import datetime

import pandas as pd

from src.features.dataloader import DataLoader
from src.models.networkx_graph import SurfaceModel

In [ ]:
# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc as get_auc
import matplotlib.pyplot as plt

# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)

# split into train/test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.5, random_state=2)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(train_X, train_y)

# predict probabilities
lr_probs = model.predict_proba(test_X)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

def plot_prec_rec_curve(y, y_hat_probs, classifier_label):
    # predict class values
    precision, recall, _ = precision_recall_curve(y, y_hat_probs)
    auc = get_auc(recall, precision)

    # summarize scores
    print(classifier_label, ': auc=%.3f' % (auc))

    # plot the precision-recall curves
    no_skill = len(y[y==1]) / len(y)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='Chance')
    plt.plot(recall, precision, marker='.', label=classifier_label)

    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')

    plt.legend() # show the legend
    plt.show()  # show the plot
    
plot_prec_rec_curve(test_y, lr_probs, "Logistic")

In [ ]:
# get tested/positive patients of range
csv_path = "./data/interim/model_data/VRE_SCREENING_DATA.csv"
risk_df = pd.read_csv(csv_path, parse_dates=["Record Date"], dtype=str)
risk_df

In [ ]:
def get_screening_pids_of_range(from_range=None, to_range=None, is_positive=False):
    risk_range_df = risk_df
    
    if from_range is not None:
        risk_range_df = risk_range_df.loc[(risk_df["Record Date"] > from_range)]
        
    if to_range is not None:
        risk_range_df = risk_range_df.loc[(risk_df["Record Date"] <= to_range)]
    
    if is_positive:
        risk_range_df = risk_range_df[risk_range_df["Pathogen Result"] != "nn"]
    
    return risk_range_df["Patient ID"].tolist()

get_screening_pids_of_range(from_range=datetime(2017, 12, 1), to_range=datetime(2018, 1, 1))

In [ ]:
# drop duplicate positive screenings to make them only show up once and not regularly (otherwise such a patient is shown as newly positive below, but the algorithm will exclude it from prediction as it is already known)
v_df_pos = risk_df[risk_df["Pathogen Result"] != "nn"]
v_df_pos["Pathogen Result"] = "pp"
v_df_pos = v_df_pos.drop_duplicates(subset=["Pathogen Result", "Patient ID"]).sort_values(by=["Record Date"])
v_df_pos

In [ ]:
risk_df = pd.concat([risk_df[risk_df["Pathogen Result"].str.contains("nn")], v_df_pos]).sort_values(by=["Record Date"])

In [ ]:
risk_df

In [ ]:
from datetime import date, datetime, timedelta

logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.INFO, datefmt='%d.%m.%Y %H:%M:%S')

now_str = datetime.now().strftime("%Y%m%d%H%M%S")

lookback_window_month_qty = 2
lookback_window = timedelta(weeks=lookback_window_month_qty * 4)
prediction_window_month_qty = 1
prediction_window = timedelta(weeks=prediction_window_month_qty * 4)

#####################################
# Validating algorithms
logging.info("Running sliding window validation...")

def datespan(start_date, end_date, delta=timedelta(days=1)):
    current_date = start_date
    while current_date < end_date:
        yield current_date
        current_date += delta

start_span = datetime(2017, 12, 1)
end_span = datetime(2021, 3, 31)

# print general stats on screenings and infections
infected_patients = get_screening_pids_of_range(is_positive=True)
print(f"Total of {len(infected_patients)} patients infected")

for start_date in datespan(start_span, end_span, timedelta(weeks=1)):
    end_date = start_date + lookback_window
    print(f"Loading data within range: {start_date} - {end_date}")
    loader = DataLoader()
    patient_data = loader.prepare_dataset(load_medications=False,
                                          load_icd_codes=False,
                                          load_chop_codes=False,
                                          load_surgeries=False,
                                          load_partners=False,
                                          from_range=start_date,
                                          to_range=end_date,
                                          load_patients_in_locations=["BH O", "BHH O"],
                                          is_verbose=False)
    
    print(f"Calculating snapshot: {start_date} - {end_date}")
    surface_graph = SurfaceModel(data_dir='./data/processed/networkx')
    surface_graph.add_network_data(patient_dict=patient_data, case_subset='relevant_case')
    surface_graph.trim_model(start_date, end_date)
    surface_graph.remove_isolated_nodes()
    surface_graph.inspect_network()
        
    infected_patients_in_lookback_range = get_screening_pids_of_range(from_range=start_date, to_range=end_date, is_positive=True)
    infected_patients_in_lookback_range = list(set(infected_patients_in_lookback_range).intersection(set(surface_graph.get_patients())))
    
    screenings_in_lookback_range = get_screening_pids_of_range(from_range=start_date, to_range=end_date, is_positive=False)
    screenings_in_lookback_range = list(set(screenings_in_lookback_range).intersection(set(surface_graph.get_patients())))
    
    infected_patients_in_prediction_range = get_screening_pids_of_range(end_date, to_range=end_date + prediction_window, is_positive=True)
    infected_patients_in_prediction_range = list(set(infected_patients_in_prediction_range).intersection(set(surface_graph.get_patients())))
    
    screenings_in_prediction_range = get_screening_pids_of_range(end_date, to_range=end_date + prediction_window, is_positive=False)
    screenings_in_prediction_range = list(set(screenings_in_prediction_range).intersection(set(surface_graph.get_patients())))
    
    
    print(f"{len(infected_patients_in_lookback_range)} / {len(screenings_in_lookback_range)} infections/screenings in lookback range {start_date.date()} - {end_date.date()}")
    print(f"{len(infected_patients_in_prediction_range)} / {len(screenings_in_prediction_range)} infections/screenings in prediction range {end_date.date()} - {(end_date + prediction_window).date()}")

    print("### Available features and targets in data ###")
    print(f"{len(surface_graph.get_positive_patients())} known positive patients in graph")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(surface_graph.get_positive_patients())))} / {len(infected_patients_in_prediction_range)} prediction patients are positive in graph (-- =0)")
    print()
    print(f"{len(set(infected_patients_in_lookback_range).intersection(set(surface_graph.get_positive_patients())))} / {len(infected_patients_in_lookback_range)} known positive patients equally in graph and in lookback range (++)") # patients without interactions are lost
    # graph and screening range numbers can differ because patients without interactions are discarded as isolated nodes -> Unpredictable nodes
    
    negative_patients_in_graph = set(surface_graph.get_patients()).difference(set(surface_graph.get_positive_patients()))
    print(f"{len(set(infected_patients_in_prediction_range).intersection(negative_patients_in_graph))} / {len(infected_patients_in_prediction_range)} infected patients in prediction range are negative in graph (++)")
    
    surface_graph.add_edge_infection(infection_distance=2)
    # calculate infection degree
    infection_degree_df = surface_graph.calculate_infection_degree()
#     print(infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Risk Status"] == "neg")].head(5))
    
    # sanity checks: are positive patients in ranking?
    infected_patients_list = infection_degree_df["Node ID"].to_list()
    print("### Are positive patients in ranking? ###")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(infected_patients_list)))} positive patients in infection degree ranking")
    print()
    
#     # print prediction based on degree ratio
#     print("### Infection Degree (ID) and degree ratio threshold ###")
#     id_prediction1 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Degree Ratio"] >= 1.0) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
#     print(f"{len(set(infected_patients).intersection(set(id_prediction1)))} / {len(infected_patients)} with {len(id_prediction1)} screenings | patients at risk predicted to be ever positive")
#     print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction1)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction1)} screenings | patients at risk predicted to be positive in range")
#     print()
       
    # print prediction based on number of infected edges 95% quartile
    print("### Infection Degree (ID) and infected edges 95% quartile threshold ###")
    id_prediction3 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Number of Infected Edges"] > infection_degree_df["Number of Infected Edges"].quantile(0.95)) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction3)))} / {len(infected_patients)} with {len(id_prediction3)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction3)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction3)} screenings | patients at risk predicted to be positive in range")
    print()
    
    # print prediction based on number of infected edges 90% quartile
    print("### Infection Degree (ID) and infected edges 90% quartile threshold ###")
    id_prediction3 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Number of Infected Edges"] > infection_degree_df["Number of Infected Edges"].quantile(0.90)) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction3)))} / {len(infected_patients)} with {len(id_prediction3)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction3)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction3)} screenings | patients at risk predicted to be positive in range")
    print()
    
    # print prediction based on number of infected edges 85% quartile
    print("### Infection Degree (ID) and infected edges 85% quartile threshold ###")
    id_prediction3 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Number of Infected Edges"] > infection_degree_df["Number of Infected Edges"].quantile(0.85)) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction3)))} / {len(infected_patients)} with {len(id_prediction3)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction3)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction3)} screenings | patients at risk predicted to be positive in range")
    print()
    
    # print prediction based on number of infected edges 75% quartile
    print("### Infection Degree (ID) and infected edges 75% quartile threshold ###")
    id_prediction3 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Number of Infected Edges"] > infection_degree_df["Number of Infected Edges"].quantile(0.75)) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction3)))} / {len(infected_patients)} with {len(id_prediction3)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction3)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction3)} screenings | patients at risk predicted to be positive in range")
    print()
    
    # print prediction based on number of infected edges median
    print("### Infection Degree (ID) and infected edges median threshold ###")
    id_prediction2 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Number of Infected Edges"] > infection_degree_df["Number of Infected Edges"].median()) & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction2)))} / {len(infected_patients)} with {len(id_prediction2)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction2)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction2)} screenings | patients at risk predicted to be positive in range")
    print()
    
    # print prediction based full infection degree df
    print("### Infection Degree (ID) (all predictions) ###")
    id_prediction4 = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Risk Status"] == "neg")]["Node ID"].to_list()
    print(f"{len(set(infected_patients).intersection(set(id_prediction4)))} / {len(infected_patients)} with {len(id_prediction4)} screenings | patients at risk predicted to be ever positive")
    print(f"{len(set(infected_patients_in_prediction_range).intersection(set(id_prediction4)))} / {len(infected_patients_in_prediction_range)} with {len(id_prediction4)} screenings | patients at risk predicted to be positive in range")
    print()
    
    y_df = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Risk Status"] == "neg")][["Node ID", "Number of Infected Edges"]]
    y_df.loc[y_df['Node ID'].isin(infected_patients_in_prediction_range), 'Target'] = 1
    y_df.loc[~y_df['Node ID'].isin(infected_patients_in_prediction_range), 'Target'] = 0
    y_hat_probs = y_df["Number of Infected Edges"].to_numpy()
    y = y_df["Target"].to_numpy()
    plot_prec_rec_curve(y, y_hat_probs, "Centrality")
    

In [ ]:
# get predicted patients with their infected edges scoring
y_df = infection_degree_df[(infection_degree_df["Node Type"] == "Patient") & (infection_degree_df["Risk Status"] == "neg")][["Node ID", "Number of Infected Edges"]]
y_df[y_df["Node ID"].isin(infected_patients_in_prediction_range)]

In [ ]:
y_df.loc[y_df['Node ID'].isin(infected_patients_in_prediction_range), 'Target'] = 1
y_df.loc[~y_df['Node ID'].isin(infected_patients_in_prediction_range), 'Target'] = 0
y_hat_probs = y_df["Number of Infected Edges"].to_numpy()
y = y_df["Target"].to_numpy()

In [ ]:
y_hat_probs

In [ ]:
plot_prec_rec_curve(y, y_hat_probs, "Centrality")

In [ ]:
y.shape

In [ ]:
y_df[y_df["Node ID"].isin(infected_patients_in_prediction_range)]

In [ ]:
infected_patients_in_prediction_range